# INVICTA Spring School
## Low-Precision Models
### By Jason K. Eshraghian (www.ncg.ucsc.edu)


[<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/snntorch_alpha_w.png?raw=true' width="200">](https://github.com/jeshraghian/snntorch/)

In [ ]:
!pip install snntorch --quiet # shift + enter

*What will I learn?*

1. Train an SNN classifier using snnTorch
2. Hardware Friendly Training
  - Weight Quantization with Brevitas
  - Stateful Quantization
3. Handling neuromorphic data with Tonic

# 1. Train an SNN Classifier using snnTorch
## 1.1 Imports


In [ ]:
# snntorch imports
import snntorch as snn
from snntorch import functional as SF

# pytorch imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# data manipulation
import numpy as np
import itertools

# plotting
import matplotlib.pyplot as plt
from IPython.display import HTML

## 1.2 Boilerplate: DataLoading the MNIST Dataset

In [ ]:
# dataloader arguments
batch_size = 128
data_path='/data/mnist'

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
## if you're on M1 or M2 GPU:
# device = torch.device("mps")

In [ ]:
# Define a transform
transform = transforms.Compose([
            transforms.Resize((28, 28)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

In [ ]:
# Create DataLoaders
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

## 1.3 Construct SNN Model

In [ ]:
# Network Architecture
num_inputs = 28*28
num_hidden = 100
num_outputs = 10

# Temporal Dynamics
num_steps = 25
beta = 0.95

In [ ]:
# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        self.lif2 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()

        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        # time-loop
        for step in range(num_steps):
          cur1 = self.fc1(x.flatten(1)) # batch: 128 x 784
          spk1, mem1 = self.lif1(cur1, mem1)
          cur2 = self.fc2(spk1)
          spk2, mem2 = self.lif2(cur2, mem2)

          # store in list
          spk2_rec.append(spk2)
          mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0) # time-steps x batch x num_out

# Load the network onto CUDA if available
net = Net().to(device)

## 1.4 Training the SNN

In [ ]:
def training_loop(model, dataloader, num_epochs=1):
  loss = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, betas=(0.9, 0.999))
  counter = 0

  # Outer training loop
  for epoch in range(num_epochs):
      train_batch = iter(dataloader)

      # Minibatch training loop
      for data, targets in train_batch:
          data = data.to(device)
          targets = targets.to(device)

          # forward pass
          model.train()
          spk_rec, _ = model(data)

          # initialize the loss & sum over time
          loss_val = torch.zeros((1), dtype=dtype, device=device)
          loss_val = loss(spk_rec.sum(0), targets) # batch x num_out

          # Gradient calculation + weight update
          optimizer.zero_grad()
          loss_val.backward()
          optimizer.step()

          # Print train/test loss/accuracy
          if counter % 10 == 0:
              print(f"Iteration: {counter} \t Train Loss: {loss_val.item()}")
          counter += 1

          if counter == 100:
            break

training_loop(net, train_loader)

In [ ]:
def measure_accuracy(model, dataloader):
  with torch.no_grad():
    model.eval()
    running_length = 0
    running_accuracy = 0

    for data, targets in iter(dataloader):
      data = data.to(device)
      targets = targets.to(device)

      # forward-pass
      spk_rec, _ = model(data)
      spike_count = spk_rec.sum(0) # batch x num_outputs
      _, max_spike = spike_count.max(1)

      # correct classes for one batch
      num_correct = (max_spike == targets).sum()

      # total accuracy
      running_length += len(targets)
      running_accuracy += num_correct

    accuracy = (running_accuracy / running_length)

    return accuracy.item()


In [ ]:
print(f"Test set accuracy: {measure_accuracy(net, test_loader)}")

### A Sanity Check

In [ ]:
def print_sample(model, dataloader, idx=0):
  with torch.no_grad():
    model.eval()

    data, targets = next(iter(dataloader))
    data = data.to(device)
    targets = targets.to(device)

    # forward-pass
    spk_rec, _ = model(data)
    spike_count = spk_rec.sum(0) # batch x num_outputs
    _, max_spike = spike_count.max(1)

    # Plot the sample
    plt.imshow(data[idx].cpu().squeeze(), cmap='gray')
    plt.title(f'Target: {targets[idx].item()}')
    plt.show()


    return

In [ ]:
print_sample(net, test_loader)

# 2. Hardware Friendly Training
## 2.1 Weight Quantization

In [ ]:
!pip install brevitas --quiet

Just replace all `nn.Linear` layers with `qnn.QuantLinear(num_inputs, num_outputs, weight_bit_width, bias)`.

In [ ]:
import brevitas.nn as qnn

# Define Network
class QuantNet(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.fc1 =
        self.lif1 =
        self.fc2 =
        self.lif2 =

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()

        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        for step in range(num_steps):
            cur1 = self.fc1(x.flatten(1))
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)

# Load the network onto CUDA if available
qnet = QuantNet().to(device)

In [ ]:
training_loop(qnet, train_loader)
print(f"Test set accuracy: {measure_accuracy(qnet, test_loader)}")

## 2.2 SQUAT: Stateful Quantization-Aware Training



In [ ]:
from snntorch.functional import quant

# Define Network
class SquatNet(nn.Module):
    def __init__(self):
        super().__init__()

        # Define state quantization parameters
        q_lif = quant.state_quant(num_bits=4, uniform=True)

        # Initialize layers
        self.fc1 =
        self.lif1 =
        self.fc2 =
        self.lif2 =

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()

        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        for step in range(num_steps):
            cur1 = self.fc1(x.flatten(1))
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)

# Load the network onto CUDA if available
sqnet = SquatNet().to(device)

In [ ]:
training_loop(sqnet, train_loader)
print(f"Test set accuracy: {measure_accuracy(sqnet, test_loader)}")

That's all folks!